In [3]:
import pandas as pd
import numpy as np
import yaml
import os
from geopy.geocoders import Nominatim
from tqdm import tqdm
from glob import glob

In [412]:
def calc_runrate(runs_scored,balls_bowled):
    
    if balls_bowled == 0:
        balls_bowled = 1
    translate_ball_to_over = (balls_bowled/6)
    #print(runs_scored,translate_ball_to_over)
    run_rate = (runs_scored)/(translate_ball_to_over)
    
    run_rate = round(run_rate)
    return run_rate

def calc_reqd_run_rate(runs_left,balls_left):
    
    if balls_left == 0:
        balls_left = 1
    translate_ball_to_over = (balls_left/6)
    req_run_rate = (runs_left)/(translate_ball_to_over)
    req_run_rate = round(req_run_rate)
    return req_run_rate

In [413]:
# match_id,venue_city,country,day,month,year,match_between,toss_winner,toss_decision,\
# current_innings_team,innings_type,over_number,batsman,non_striker,bowler,extras,total_runs,batsman_runs,\
# extras_type,current_run_rate,req_run_rate,\
# out,player_out,out_type,dismissed_by_fielder,winner_team,winner_method,winner_num,method_if_any,\
# super_over,super_over_winner_team,potm

df_data = []

def get_country(venue):
    
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.geocode(venue,language="en")
    country = location.address.split(",")[-1].strip()
    return country


def create_csv(folder_path):
    
   
    for file in tqdm(os.listdir(folder_path)):
        #print(file)
        try:
            with open(folder_path+file) as stream:
                data = yaml.safe_load(stream)

                ####### Static Data#####
                match_id = file.split(".")[0]
                try:
                    venue_city = data["info"]["city"]
                except:    
                    venue_city = data["info"]["venue"]
                #print(venue_city)
                try:
                    country = get_country(venue_city)
                except:
                    country = "None"
                #print(file,venue_city,country)
                #print(data["info"]["dates"][0],str(data["info"]["dates"][0]).split("-"))
                day,month,year = str(data["info"]["dates"][0]).split("-")
                if "eliminator" in list(data["info"]["outcome"].keys()):
                    #super_over = "Yes"
                    super_over_winner_team = data["info"]["outcome"]["eliminator"]
                    winner_team = "Tie"
                    winner_method = "Tie"
                    winner_num = "Tie"

                elif "result" in list(data["info"]["outcome"].keys()):
                    if data["info"]["outcome"] == "no result":

                        winner_team = "No Result"
                        winner_method = "No Result"
                        winner_num = "No Result"
                        super_over_winner_team = "None"

                else:
                    winner_team = data["info"]["outcome"]["winner"]
                    if "by" in list(data["info"]["outcome"].keys()):
                        winner_method = list(data["info"]["outcome"]["by"].keys())[0]
                        winner_num = list(data["info"]["outcome"]["by"].values())[0]
                    else:
                        ###Awarded Methods###
                        winner_method = data["info"]["outcome"]["method"]
                        winner_num = "None"
                    #super_over = "No"
                    super_over_winner_team = "None"

                match_between = data["info"]["teams"][0] + " vs " + data["info"]["teams"][1]
                toss_winner = data["info"]["toss"]["winner"]
                toss_decision = data["info"]["toss"]["decision"]
                try:
                    potm = "/".join((data["info"]["player_of_match"]))
                except:
                    potm = "No Data"

                if "method" in data["info"]["outcome"].keys():
                    method_if_any = data["info"]["outcome"]["method"]
                else:
                    method_if_any = "None"
                #############################
                
                target = 0
                curr_innings_number = 0
                
                for inn in data["innings"]:
                    innings = list(inn.values())[0]
                    #print(innings)
                    #break
                    #runs_set = 0
                    
                    balls_bowled = 0
                    balls_rem = 120
                    
                    runs_chase = 0
                    
                    curr_innings_number = curr_innings_number + 1
                    innings_number = curr_innings_number
                    if innings_number%2 == 0:
                        innings_type = "chasing"
                        runs_set = target
                    else:
                        innings_type = "score_setter"
                        target = 0
                        
                    
                    current_innings_team = innings["team"]
                    if "super over" in list(inn.keys())[0].lower():
                        super_over = "Yes"
                    else:
                        super_over = "No"
                        
                    curr_score,curr_wickets = 0,0
                    for delivery in list(inn.values())[0]["deliveries"]:
                        #print(delivery)
                        over_number = list(delivery.keys())[0]
                        over_values = list(delivery.values())[0]
                        batsman = over_values["batsman"]
                        non_striker = over_values["non_striker"]
                        bowler = over_values["bowler"]
                        extras,total_runs,batsman_runs = over_values["runs"]["extras"],over_values["runs"]["total"],over_values["runs"]["batsman"]
                        if extras > 0:
                            extras_type = list(over_values["extras"].keys())[0]
                            
                        else:
                            extras_type = "None"
                            if innings_type == "score_setter":
                                balls_bowled = balls_bowled + 1
                            else:
                                balls_bowled = balls_bowled + 1
                                balls_rem = balls_rem - 1
                                
                        curr_score = curr_score + total_runs
                        
                            
                        if innings_type == "score_setter":
                            target = target + total_runs
                            current_run_rate = calc_runrate(target,balls_bowled)
                            reqd_run_rate = 0
                        else:
                            #print("HI")
                            runs_chase = runs_chase + total_runs
                            runs_remaining = runs_set - runs_chase
                            if runs_remaining < 0:
                                runs_remaining = 0
                           # print(runs_remaining,balls_rem)
                            reqd_run_rate = calc_reqd_run_rate(runs_remaining,balls_rem)
                            #print(reqd_run_rate)
                            current_run_rate = calc_runrate(runs_chase,balls_bowled)
                            

                        if "wicket" in over_values.keys():
                            out = "Yes"
                            player_out = over_values["wicket"]["player_out"]
                            out_type = over_values["wicket"]["kind"]
                            if "fielders" in over_values["wicket"]:
                                dismissed_by_fielder = "/".join((over_values["wicket"]["fielders"]))
                            else:
                                dismissed_by_fielder = "None"
                            curr_wickets = curr_wickets + 1
                        else:
                            out = "No"
                            player_out = "None"
                            out_type = "None"
                            dismissed_by_fielder = "None"
                            
                        
                        
                        #print(reqd_run_rate)

                        df_data.append([match_id,venue_city,country,day,month,year,match_between,toss_winner,toss_decision,\
                       current_innings_team,innings_type,over_number,batsman,non_striker,bowler,extras,total_runs,batsman_runs,\
                                    extras_type,curr_score,curr_wickets,current_run_rate,reqd_run_rate,\
                        out,player_out,out_type,dismissed_by_fielder,winner_team,winner_method,winner_num,method_if_any,\
                                        super_over,super_over_winner_team,potm])


        #return df_data
        except TypeError as e:
            print("passed")
            pass

In [414]:
create_csv("/home/shivargha/cricket_analytics/T20_cricket/t20s_part2/")
df = pd.DataFrame(df_data,
                  columns=["matchid","venue","country","date","month","year","match_between",
                           "toss_winner","toss_decision",\
                          "current_innings","innings_type","over","batsman","non_striker","bowler","extras",\
                           "total_runs","batsman_runs",\
                          "extras_type","curr_score","curr_wickers","current_run_rate","req_run_rate","out","out_batsman","dismissal_type","dismissed_by_fielder",\
                          "winner","winner_method","win_by","method_if_any","super_over","super_over_winner","potm" 
                          ])

 92%|█████████████████████████████████████████████████████████████████████████████████████████████        | 1603/1740 [23:41<01:25,  1.60it/s]

passed


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1740/1740 [25:40<00:00,  1.13it/s]


In [415]:
print("Maximum Batsman Runs:",max(df["batsman_runs"]))

Maximum Batsman Runs: 7


In [416]:
##Run Out/Obstructing the field  Considered here as bowler's wicker for simplicity###
non_bowler_type = ["None","retired hurt","retired out","retired not out"]
bowler_out_type = []
for out in df["dismissal_type"].unique():
    if out not in non_bowler_type:
        bowler_out_type.append(out)
print(bowler_out_type)

['lbw', 'run out', 'caught', 'bowled', 'caught and bowled', 'stumped', 'hit wicket', 'obstructing the field']


In [417]:
bowler_out_type

['lbw',
 'run out',
 'caught',
 'bowled',
 'caught and bowled',
 'stumped',
 'hit wicket',
 'obstructing the field']

In [418]:
from math import floor,ceil

In [419]:
batsmans = []
bowlers = []
match_ids = []

batsman_score = []
balls_played_by_batsman = []
batsman_strike_rate = []

runs_conceded_by_bowler = []
balls_bowled_by_bowler = []
wickets_taken_by_bowler = []
bowler_economy = []

In [420]:
def calc_batsman_strike_rate(runs,balls_played):
    
    strike_rate = (runs/balls_played) *100
    return strike_rate

def calc_economy_rate(runs_given,balls_bowled):
    
    translate_ball_to_over = (balls_bowled/6)
    economy = (runs_given)/(translate_ball_to_over)
    economy = round(economy)
    return economy
    
for row in tqdm(range(len(df))):
    #if row == 20:
     #   break
    #print(row)
    curr_match_id = df.iloc[row]["matchid"]
    match_ids.append(curr_match_id)
    
    striker = df.iloc[row]["batsman"]
    batsmans.append(striker)
    bowler = df.iloc[row]["bowler"]
    bowlers.append(bowler)
    
    ###############################batsman statistics#####################################################
    batsman_runs = df.iloc[row]["batsman_runs"]
    ####updating batsman runs####
    #print(len(batsman_score),len(match_ids))
    #print(striker,bowler,batsman_runs)
    if (len(batsman_score) == 0) or (curr_match_id != match_ids[-2]):
        ###These conditions required for initial updations, or for opening batsman in a new match###
        batsman_score.append(batsman_runs)
        balls_played_by_batsman.append(1)
        batsman_strike_rate.append(calc_batsman_strike_rate(batsman_runs,1))
    
    ##### conditions to add up the previous score and the present runs made by batsman###
    elif striker == batsmans[-2]:
    
        pres_score = batsman_score[-1] + batsman_runs
        batsman_score.append(pres_score)
        pres_balls_played = balls_played_by_batsman[-1] + 1
        balls_played_by_batsman.append(pres_balls_played)
        
        batsman_strike_rate.append(calc_batsman_strike_rate(pres_score,pres_balls_played))
        
        
    elif striker == batsmans[-3]:
        
        pres_score = batsman_score[-2] + batsman_runs
        batsman_score.append(pres_score)
        pres_balls_played = balls_played_by_batsman[-2] + 1
        balls_played_by_batsman.append(pres_balls_played)
        
        batsman_strike_rate.append(calc_batsman_strike_rate(pres_score,pres_balls_played))
    ############################################################################    
        
    ##condition used to take into consideration for debut of batsman,
    ## first time updation of runs, when a new batsman comes to crease###
    elif striker not in batsmans[:-1]:
        batsman_score.append(batsman_runs)
        balls_played_by_batsman.append(1)
        
        batsman_strike_rate.append(calc_batsman_strike_rate(batsman_runs,1))
    #####################################################################
    
    elif striker in batsmans:
        
        
        ###condition when a batsman is retired hurt and comes out to bat again later####
        if striker not in batsmans[-3:-1]:
            search_batsmans = batsmans[:-1]
            last_index_val = len(search_batsmans) - search_batsmans[::-1].index(striker) - 1 ##get the latest index value of this
            striker_match_id = match_ids[last_index_val]                                  ##batsman
            if striker_match_id == curr_match_id:
                ###this solves the retire hurt bastman who comes back again issue##
                pres_score = batsman_score[last_index_val] + batsman_runs
                pres_balls_played = balls_played_by_batsman[last_index_val] + 1
                batsman_score.append(pres_score)
                balls_played_by_batsman.append(pres_balls_played)
                
                batsman_strike_rate.append(calc_batsman_strike_rate(pres_score,pres_balls_played))
                
            elif striker_match_id != curr_match_id: ##condition when its a new match##
                batsman_score.append(batsman_runs)
                balls_played_by_batsman.append(1)
                batsman_strike_rate.append(calc_batsman_strike_rate(batsman_runs,1))
                
        ###### condition when a batsman comes out to bat in the innings such that,
        ##### he previously ended the match as the lst batsman, or retired hurt for only one or two balls ###
        elif striker in batsmans[-3:-1]:
            search_batsmans = batsmans[:-1]
            last_index_val = len(search_batsmans) - search_batsmans[::-1].index(striker) - 1
            striker_match_id = match_ids[last_index_val]
            if striker_match_id == curr_match_id:
                pres_score = batsman_score[last_index_val] + batsman_runs
                pres_balls_played = balls_played_by_batsman[last_index_val] + 1
                batsman_score.append(pres_score)
                balls_played_by_batsman.append(pres_balls_played)
                batsman_strike_rate.append(calc_batsman_strike_rate(pres_score,pres_balls_played))
                
            elif striker_match_id != curr_match_id:
                batsman_score.append(batsman_runs)
                balls_played_by_batsman.append(1)
                batsman_strike_rate.append(calc_batsman_strike_rate(batsman_runs,1))
    ###################################################################################################        
    
    ################################# bowler statistics ####################################################
    out,out_type = df.iloc[row]["out"],df.iloc[row]["dismissal_type"]
    if (len(runs_conceded_by_bowler) == 0) or (curr_match_id != match_ids[-2]):
        runs_conceded_by_bowler.append(batsman_runs)
        balls_bowled_by_bowler.append(1)
        if out == "Yes":
            if out_type in bowler_out_type:
                wickets_taken_by_bowler.append(1)
            else:
                wickets_taken_by_bowler.append(0)

        else:
            wickets_taken_by_bowler.append(0)
            
        bowler_economy.append(calc_economy_rate(batsman_runs,1))
            
    elif bowler == bowlers[-2]:
    
        pres_conceded = runs_conceded_by_bowler[-1] + batsman_runs
        runs_conceded_by_bowler.append(pres_conceded)
        pres_balls_bowled = balls_bowled_by_bowler[-1] + 1
        balls_bowled_by_bowler.append(pres_balls_bowled)
        if out == "Yes":
            if out_type in bowler_out_type:
                pres_wickets = wickets_taken_by_bowler[-1] + 1
                wickets_taken_by_bowler.append(pres_wickets)
            else:
                pres_wickets = wickets_taken_by_bowler[-1]
                wickets_taken_by_bowler.append(pres_wickets)
        else:
            pres_wickets = wickets_taken_by_bowler[-1]
            wickets_taken_by_bowler.append(pres_wickets)
            
        bowler_economy.append(calc_economy_rate(pres_conceded,pres_balls_bowled))
        
    elif bowler != bowlers[-2]:
        search_bowlers = bowlers[:-1]
        if bowler in search_bowlers:
#             bowlers = bowlers[:-1]
            last_index_val = len(search_bowlers) - search_bowlers[::-1].index(bowler) - 1 ##get the latest index value of this
            bowler_match_id = match_ids[last_index_val]                                  ##bowler
            if bowler_match_id == curr_match_id:
                pres_conceded = runs_conceded_by_bowler[last_index_val] + batsman_runs
                runs_conceded_by_bowler.append(pres_conceded)
                pres_balls_bowled = balls_bowled_by_bowler[last_index_val] + 1
                balls_bowled_by_bowler.append(pres_balls_bowled)
                #print(last_index_val)
                if out == "Yes":
                    if out_type in bowler_out_type:
                        pres_wickets = wickets_taken_by_bowler[last_index_val] + 1
                        wickets_taken_by_bowler.append(pres_wickets)
                    else:
                        pres_wickets = wickets_taken_by_bowler[last_index_val]
                        wickets_taken_by_bowler.append(pres_wickets)
                        
                else:
                    pres_wickets = wickets_taken_by_bowler[last_index_val]
                    wickets_taken_by_bowler.append(pres_wickets)
                    
                bowler_economy.append(calc_economy_rate(pres_conceded,pres_balls_bowled))
                
            elif bowler_match_id != curr_match_id: ##condition when its a new match##
                runs_conceded_by_bowler.append(batsman_runs)
                balls_bowled_by_bowler.append(1)
                if out == "Yes":
                    if out_type in bowler_out_type:
                        wickets_taken_by_bowler.append(1)
                    else:
                        wickets_taken_by_bowler.append(0)
                else:
                    wickets_taken_by_bowler.append(0)
                    
                bowler_economy.append(calc_economy_rate(batsman_runs,1))
        else:
            runs_conceded_by_bowler.append(batsman_runs)
            balls_bowled_by_bowler.append(1)
            if out == "Yes":
                if out_type in bowler_out_type:
                    wickets_taken_by_bowler.append(1)
                else:
                    wickets_taken_by_bowler.append(0)
            else:
                wickets_taken_by_bowler.append(0)
                
            bowler_economy.append(calc_economy_rate(batsman_runs,1))
            
    ###########################################################################################################

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 396981/396981 [47:01<00:00, 140.68it/s]


In [421]:
# batsmans ,bowlers, match_ids ,batsman_score ,balls_played_by_batsman 
# batsman_strike_rate ,runs_conceded_by_bowler , balls_bowled_by_bowler 
# wickets_taken_by_bowler, bowler_economy 

df["batsman_score"] = batsman_score
df["balls_faced_batsman"] = balls_played_by_batsman
df["batsman_strike_rate"] = batsman_strike_rate
df["runs_conceded_by_bowler"] = runs_conceded_by_bowler
df["balls_bowled_bowler"] = balls_bowled_by_bowler
df["wickets_by_bower"] = wickets_taken_by_bowler
df["bowler_economy"] = bowler_economy

In [422]:
df.columns

Index(['matchid', 'venue', 'country', 'date', 'month', 'year', 'match_between',
       'toss_winner', 'toss_decision', 'current_innings', 'innings_type',
       'over', 'batsman', 'non_striker', 'bowler', 'extras', 'total_runs',
       'batsman_runs', 'extras_type', 'curr_score', 'curr_wickers',
       'current_run_rate', 'req_run_rate', 'out', 'out_batsman',
       'dismissal_type', 'dismissed_by_fielder', 'winner', 'winner_method',
       'win_by', 'method_if_any', 'super_over', 'super_over_winner', 'potm',
       'batsman_score', 'balls_faced_batsman', 'batsman_strike_rate',
       'runs_conceded_by_bowler', 'balls_bowled_bowler', 'wickets_by_bower',
       'bowler_economy'],
      dtype='object')

In [423]:
match_id_list = []
batsman_names = []
balls_faced_list = []
batsman_propotion_list = []
batsman_actual_count_list = []

counter = 0
for row in tqdm(range(len(df))):
    
    match_id = df.iloc[row]["matchid"]
    match_id_list.append(match_id)
    ##########Batsman Statistics############
    striker = df.iloc[row]["batsman"]
    batsman_names.append(striker)
    balls_faced = df.iloc[row]["balls_faced_batsman"]
    balls_faced_list.append(balls_faced)
    batsman_runs = df.iloc[row]["batsman_runs"] 
    
    #print(batsman_runs)
    
#     if row == 4:
#         break
    #print("12",counter)
    if (counter == 0) or (match_id != match_id_list[-2]):
        ###new entry/initialiser###
        dist_dict_prop = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0}
        dist_dict_count = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0}
        run_count = 1
        dist_dict_prop[batsman_runs] = round(run_count/1,4)
        dist_dict_count[batsman_runs] = run_count
        batsman_propotion_list.append(dist_dict_prop)
        batsman_actual_count_list.append(dist_dict_count)
        
        #print(dist_dict_count,dist_dict_prop)
        counter = counter + 1
       
    #print(counter)
    elif striker in batsman_names[:-1]:
        
        #print("HI")
        search_batsman = batsman_names[:-1]
        last_index_val = len(search_batsman) - search_batsman[::-1].index(striker) - 1
        striker_match_id = match_id_list[last_index_val] 
        
        if match_id == striker_match_id:     
            #print(batsman_runs)
            prev_dist_dict_prop = batsman_propotion_list[last_index_val]
            prev_dist_dict_count = batsman_actual_count_list[last_index_val]
            
            new_dist_dict_prop = prev_dist_dict_prop.copy()
            new_dist_dict_count = prev_dist_dict_count.copy()
            
            
            balls_faced_pres = balls_faced_list[last_index_val] + 1 
            
            prev_count = prev_dist_dict_count[batsman_runs]
            new_count = prev_count + 1
            
            new_dist_dict_count[batsman_runs] = new_count
            
            new_dist_dict_prop[batsman_runs] = round((new_count)/(balls_faced_pres),4)
            
            for i in new_dist_dict_prop:
                if i != batsman_runs:
                    count_run = new_dist_dict_count[i]
                    new_dist_dict_prop[i] = round(count_run/balls_faced_pres,4)
            #print("balls",balls_faced_pres) 
            #print(prev_dist_dict_count,prev_dist_dict_prop)
            batsman_propotion_list.append(new_dist_dict_prop)
            batsman_actual_count_list.append(new_dist_dict_count)
        
        elif match_id != striker_match_id:
            dist_dict_prop = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0}
            dist_dict_count = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0}
            run_count = 1
            dist_dict_prop[batsman_runs] = round(run_count/1,4)
            dist_dict_count[batsman_runs] = run_count
            batsman_propotion_list.append(dist_dict_prop)
            batsman_actual_count_list.append(dist_dict_count)
            
    elif striker not in batsman_names[:-1]:
        
        dist_dict_prop = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0}
        dist_dict_count = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0}
        run_count = 1
        dist_dict_prop[batsman_runs] = round(run_count/1,4)
        dist_dict_count[batsman_runs] = run_count
        batsman_propotion_list.append(dist_dict_prop)
        batsman_actual_count_list.append(dist_dict_count)

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 396981/396981 [1:46:33<00:00, 62.09it/s]


In [424]:
match_id_list = []
bowler_names = []
balls_bowled_list = []
bowler_propotion_list = []
bowler_actual_count_list = []
############# highest batsman runs is 7, hence taking 8 as wicket#####

counter = 0
for row in tqdm(range(len(df))):
    
    match_id = df.iloc[row]["matchid"]
    match_id_list.append(match_id)
    ##########bowler Statistics############
    bowler = df.iloc[row]["bowler"]
    bowler_names.append(bowler)
    balls_bowled = df.iloc[row]["balls_bowled_bowler"]
    balls_bowled_list.append(balls_bowled)
    out,out_type = df.iloc[row]["out"],df.iloc[row]["dismissal_type"]
    if out_type in bowler_out_type:
        out = "Yes"
    else:
        out = "No"
        
    batsman_runs = df.iloc[row]["batsman_runs"] 
    #print(batsman_runs)
#     if row == 6:
#         break
    if (counter == 0) or (match_id != match_id_list[-2]):
        ###new entry/initialiser###
        if out == "Yes":
            batsman_runs = 8
        count_num = 1
        dist_dict_prop = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0}
        dist_dict_count = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0}
        dist_dict_prop[batsman_runs] = round(count_num/1,4)
        dist_dict_count[batsman_runs] = count_num
        bowler_propotion_list.append(dist_dict_prop)
        bowler_actual_count_list.append(dist_dict_count)
        counter = counter + 1
        
       
    elif bowler in bowler_names[:-1]:
        
        search_bowler = bowler_names[:-1]
        last_index_val = len(search_bowler) - search_bowler[::-1].index(bowler) - 1
        bowler_match_id = match_id_list[last_index_val] 
        
        if match_id == bowler_match_id:
            
            if out == "Yes":
                batsman_runs = 8
            
            prev_dist_dict_prop = bowler_propotion_list[last_index_val]
            prev_dist_dict_count = bowler_actual_count_list[last_index_val]
            
            new_dist_dict_prop = prev_dist_dict_prop.copy()
            new_dist_dict_count = prev_dist_dict_count.copy()
            
            balls_bowled_pres = balls_bowled_list[last_index_val] + 1 
            
            prev_count = prev_dist_dict_count[batsman_runs]
            new_count = prev_count + 1
            
            
                
            new_dist_dict_count[batsman_runs] = new_count
            new_dist_dict_prop[batsman_runs] = round((new_count)/(balls_bowled_pres),4)
            #print(new_count,balls_faced_pres,round((new_count)/(balls_bowled_pres),4))
            
            #prev_dist_dict[batsman_runs] = round(batsman_runs/balls_bowled_pres,2)
            
            for i in new_dist_dict_prop:
                if i != batsman_runs:
                    count_run = new_dist_dict_count[i]
                    new_dist_dict_prop[i] = round((count_run)/(balls_bowled_pres),4)
                    
            bowler_propotion_list.append(new_dist_dict_prop)
            bowler_actual_count_list.append(new_dist_dict_count)
        
        elif match_id != bowler_match_id:
            if out == "Yes":
                batsman_runs = 8
            run_count = 1
            dist_dict_prop = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0}
            dist_dict_count = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0}
            dist_dict_prop[batsman_runs] = round(run_count/1,4)
            dist_dict_count[batsman_runs] = run_count
            bowler_propotion_list.append(dist_dict_prop)
            bowler_actual_count_list.append(dist_dict_count)
            
            
    elif bowler not in bowler_names[:-1]:
        if out == "Yes":
            batsman_runs = 8
        run_count = 1
        dist_dict_prop = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0}
        dist_dict_count = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0}
        dist_dict_prop[batsman_runs] = round(run_count/1,4)
        dist_dict_count[batsman_runs] = run_count
        bowler_propotion_list.append(dist_dict_prop)
        bowler_actual_count_list.append(dist_dict_count)
        

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 396981/396981 [1:49:10<00:00, 60.60it/s]


In [425]:
batsman_prop0 = []
batsman_prop1 = []
batsman_prop2 = []
batsman_prop3 = []
batsman_prop4 = []
batsman_prop5 = []
batsman_prop6 = []
batsman_prop7 = []
for index in tqdm(range(len(batsman_propotion_list))):
    dict1 = batsman_propotion_list[index]
    for i in dict1:
        if i == 0:
            batsman_prop0.append(dict1[i])
        elif i == 1:
            batsman_prop1.append(dict1[i])
        elif i == 2:
            batsman_prop2.append(dict1[i])
        elif i == 3:
            batsman_prop3.append(dict1[i])
        elif i == 4:
            batsman_prop4.append(dict1[i])
        elif i == 5:
            batsman_prop5.append(dict1[i])
        elif i == 6:
            batsman_prop6.append(dict1[i])
        elif i == 7:
            batsman_prop7.append(dict1[i])
            
    

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 396981/396981 [00:00<00:00, 515178.83it/s]


In [426]:
###########
df["batsman_prop0"] = batsman_prop0
df["batsman_prop1"] = batsman_prop1
df["batsman_prop2"] = batsman_prop2
df["batsman_prop3"] = batsman_prop3
df["batsman_prop4"] = batsman_prop4
df["batsman_prop5"] = batsman_prop5
df["batsman_prop6"] = batsman_prop6
df["batsman_prop7"] = batsman_prop7

In [427]:
bowler_prop0 = []
bowler_prop1 = []
bowler_prop2 = []
bowler_prop3 = []
bowler_prop4 = []
bowler_prop5 = []
bowler_prop6 = []
bowler_prop7 = []
bowler_prop8 = []

for index in tqdm(range(len(bowler_propotion_list))):
    dict1 = bowler_propotion_list[index]
    for i in dict1:
        if i == 0:
            bowler_prop0.append(dict1[i])
        elif i == 1:
            bowler_prop1.append(dict1[i])
        elif i == 2:
            bowler_prop2.append(dict1[i])
        elif i == 3:
            bowler_prop3.append(dict1[i])
        elif i == 4:
            bowler_prop4.append(dict1[i])
        elif i == 5:
            bowler_prop5.append(dict1[i])
        elif i == 6:
            bowler_prop6.append(dict1[i])
        elif i == 7:
            bowler_prop7.append(dict1[i])
        elif i == 8:
            bowler_prop8.append(dict1[i])
            

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 396981/396981 [00:00<00:00, 450180.69it/s]


In [428]:
df["bowler_prop0"] = bowler_prop0
df["bowler_prop1"] = bowler_prop1
df["bowler_prop2"] = bowler_prop2
df["bowler_prop3"] = bowler_prop3
df["bowler_prop4"] = bowler_prop4
df["bowler_prop5"] = bowler_prop5
df["bowler_prop6"] = bowler_prop6
df["bowler_prop7"] = bowler_prop7
df["bowler_prop8"] = bowler_prop8

In [429]:
df.columns

Index(['matchid', 'venue', 'country', 'date', 'month', 'year', 'match_between',
       'toss_winner', 'toss_decision', 'current_innings', 'innings_type',
       'over', 'batsman', 'non_striker', 'bowler', 'extras', 'total_runs',
       'batsman_runs', 'extras_type', 'curr_score', 'curr_wickers',
       'current_run_rate', 'req_run_rate', 'out', 'out_batsman',
       'dismissal_type', 'dismissed_by_fielder', 'winner', 'winner_method',
       'win_by', 'method_if_any', 'super_over', 'super_over_winner', 'potm',
       'batsman_score', 'balls_faced_batsman', 'batsman_strike_rate',
       'runs_conceded_by_bowler', 'balls_bowled_bowler', 'wickets_by_bower',
       'bowler_economy', 'batsman_prop0', 'batsman_prop1', 'batsman_prop2',
       'batsman_prop3', 'batsman_prop4', 'batsman_prop5', 'batsman_prop6',
       'batsman_prop7', 'bowler_prop0', 'bowler_prop1', 'bowler_prop2',
       'bowler_prop3', 'bowler_prop4', 'bowler_prop5', 'bowler_prop6',
       'bowler_prop7', 'bowler_prop8'],
    

In [430]:
#####Batsman Indexes############
explosivity_dict = {}
running_rating_dict = {}
power_play_explosivity = {}
end_over_explosivity = {}
batting_experience_rating = {}


batsman_names = list(df["batsman"].unique())
for batsman in tqdm(batsman_names):
    
    #batsman = df.iloc[row]["batsman"]
    df_batsman = df[df["batsman"] == batsman]
    
    batsman_4s,batsman_6s = df_batsman[df_batsman["batsman_runs"]==4],df_batsman[df_batsman["batsman_runs"]==6]
    batsman_1s,batsman_2s,batsman_3s = df_batsman[df_batsman["batsman_runs"]==1],df_batsman[df_batsman["batsman_runs"]==2],\
                                        df_batsman[df_batsman["batsman_runs"] == 3]
    ######explosivity rating####
    explosivity_rating = (len(batsman_4s)+len(batsman_6s))/(len(df_batsman))
    explosivity_dict[batsman] = explosivity_rating
    ####### Running Rating #######
    running_rating = (len(batsman_1s)+len(batsman_2s)+len(batsman_3s))/(len(df_batsman))
    running_rating_dict[batsman] = running_rating
    
    #########Power Play Explosivity##########
    df_batsman_powerplay = df_batsman[df_batsman["over"] <= 6.0]
    if len(df_batsman_powerplay) > 0 :
        batsman_4s,batsman_6s = df_batsman_powerplay[df_batsman_powerplay["batsman_runs"]==4],df_batsman_powerplay[df_batsman_powerplay["batsman_runs"]==6]
        pp_rating = (len(batsman_4s)+len(batsman_6s))/(len(df_batsman_powerplay))
        power_play_explosivity[batsman] = pp_rating
    else:
        power_play_explosivity[batsman] = 0
    
    ##########End Over Explosivity########
    df_batsman_end = df_batsman[df_batsman["over"] >= 15.0]
    if len(df_batsman_end) > 0 :
        batsman_4s,batsman_6s = df_batsman_end[df_batsman_end["batsman_runs"]==4],df_batsman_end[df_batsman_end["batsman_runs"]==6]
        eo_rating = (len(batsman_4s)+len(batsman_6s))/(len(df_batsman_end))
        end_over_explosivity[batsman] = eo_rating
    else:
        end_over_explosivity[batsman] = 0   

    ######Experience Rating ##############
    matches = len(list(df_batsman["matchid"].unique()))
    if matches > 50:
        experience = "High"
    elif ( 10 < matches < 50):
        experience = "Medium"
    elif matches < 10:
        experience = "Low"
        
    batting_experience_rating[batsman] = experience
    
    
    
################Bowler Indexes###########################
wicket_taking_rating = {}
bowling_consistency_rating = {}
bowler_experience_rating = {}

bowler_names = list(df["bowler"].unique())
for bowler in tqdm(bowler_names):
    
    df_bowler = df[df["bowler"] == bowler]
    num_outs = df_bowler[(df_bowler["out"] == "Yes")]
    wicket_taking_rating[bowler] = len(num_outs)/len(df_bowler)
    
    num_runs = sum(df_bowler["batsman_runs"])
    try:
        bowling_consistency_rating[bowler] = num_runs/len(num_outs)
    except:
        bowling_consistency_rating[bowler] = num_runs/1
    
    matches = len(list(df_bowler["matchid"].unique()))
    if matches > 50:
        experience = "High"
    elif ( 10 < matches < 50):
        experience = "Medium"
    elif matches < 10:
        experience = "Low"
        
    bowler_experience_rating[bowler] = experience

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 2011/2011 [01:14<00:00, 27.15it/s]


In [431]:
# explosivity_dict = {}
# running_rating_dict = {}
# power_play_explosivity = {}
# end_over_explosivity = {}
# batting_experience_rating = {}

# wicket_taking_rating = {}
# bowling_consistency_rating = {}
# bowler_experience_rating = {}

explosivity_list = []
running_rating_list = []
power_play_explosivity_list = []
end_over_explosivity_list = []
batting_experience_rating_list = []

wicket_taking_rating_list = []
bowling_consistency_rating_list = []
bowler_experience_rating_list = []


for row in tqdm(range(len(df))):
    
    batsman = df.iloc[row]["batsman"]
    bowler = df.iloc[row]["bowler"]
    
    explosivity_list.append(explosivity_dict[batsman])
    running_rating_list.append(running_rating_dict[batsman])
    power_play_explosivity_list.append(power_play_explosivity[batsman])
    end_over_explosivity_list.append(end_over_explosivity[batsman])
    batting_experience_rating_list.append(batting_experience_rating[batsman])
    
    wicket_taking_rating_list.append(wicket_taking_rating[bowler])
    bowling_consistency_rating_list.append(bowling_consistency_rating[bowler])
    bowler_experience_rating_list.append(bowler_experience_rating[bowler])

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 396981/396981 [01:25<00:00, 4629.60it/s]


In [432]:
df["explosivity"]  = explosivity_list
df["running_rating"] = running_rating_list
df["power_play_rating"] = power_play_explosivity_list
df["end_over_explosivity"] = end_over_explosivity_list
df["batting_experience"] = batting_experience_rating_list

df["wicket_taking_rating"] = wicket_taking_rating_list
df["bowling_consistency_rating"] = bowling_consistency_rating_list 
df["bowler_experience"] = bowler_experience_rating_list

In [492]:
bowling_team_name = []
for row in tqdm(range(len(df))):
    match_between = [i.strip() for i in df.iloc[row]['match_between'].split("vs")]
    current_inn = df.iloc[row]["current_innings"]
    match_between.remove(current_inn)
    bowl_team = match_between[0]
    bowling_team_name.append(bowl_team)

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 396981/396981 [01:50<00:00, 3586.95it/s]


In [494]:
df["bowling_team_name"] = bowling_team_name

In [24]:
df.to_csv("/home/shivargha/cricket_analytics/T20_cricket/t20is3.csv",index=False)

In [2]:
import pandas as pd
#############Data Analysis#############
df2 = pd.read_csv("/home/shivargha/cricket_analytics/T20_cricket/t20is3.csv")
df2.head()

/tmp/ipykernel_11106/624237119.py:3: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("/home/shivargha/cricket_analytics/T20_cricket/t20is3.csv")


,matchid,venue,country,date,month,year,match_between,toss_winner,toss_decision,current_innings,...,bowler_prop8,explosivity,running_rating,power_play_rating,end_over_explosivity,batting_experience,wicket_taking_rating,bowling_consistency_rating,bowler_experience,bowling_team_name
0,1140069,Karachi,Pakistan,2018,4,1,Pakistan vs West Indies,West Indies,field,Pakistan,...,0.0,0.169118,0.420956,0.189509,0.195402,High,0.052496,19.016393,High,West Indies
1,1140069,Karachi,Pakistan,2018,4,1,Pakistan vs West Indies,West Indies,field,Pakistan,...,0.0,0.169118,0.420956,0.189509,0.195402,High,0.052496,19.016393,High,West Indies
2,1140069,Karachi,Pakistan,2018,4,1,Pakistan vs West Indies,West Indies,field,Pakistan,...,0.0,0.169118,0.420956,0.189509,0.195402,High,0.052496,19.016393,High,West Indies
3,1140069,Karachi,Pakistan,2018,4,1,Pakistan vs West Indies,West Indies,field,Pakistan,...,0.0,0.169118,0.420956,0.189509,0.195402,High,0.052496,19.016393,High,West Indies
4,1140069,Karachi,Pakistan,2018,4,1,Pakistan vs West Indies,West Indies,field,Pakistan,...,0.0,0.169118,0.420956,0.189509,0.195402,High,0.052496,19.016393,High,West Indies


In [3]:
df2.columns

Index(['matchid', 'venue', 'country', 'date', 'month', 'year', 'match_between',
       'toss_winner', 'toss_decision', 'current_innings', 'innings_type',
       'over', 'batsman', 'non_striker', 'bowler', 'extras', 'total_runs',
       'batsman_runs', 'extras_type', 'curr_score', 'curr_wickers',
       'current_run_rate', 'req_run_rate', 'out', 'out_batsman',
       'dismissal_type', 'dismissed_by_fielder', 'winner', 'winner_method',
       'win_by', 'method_if_any', 'super_over', 'super_over_winner', 'potm',
       'batsman_score', 'balls_faced_batsman', 'batsman_strike_rate',
       'runs_conceded_by_bowler', 'balls_bowled_bowler', 'wickets_by_bower',
       'bowler_economy', 'batsman_prop0', 'batsman_prop1', 'batsman_prop2',
       'batsman_prop3', 'batsman_prop4', 'batsman_prop5', 'batsman_prop6',
       'batsman_prop7', 'bowler_prop0', 'bowler_prop1', 'bowler_prop2',
       'bowler_prop3', 'bowler_prop4', 'bowler_prop5', 'bowler_prop6',
       'bowler_prop7', 'bowler_prop8', 'expl

In [4]:
df4 = df2[["venue","current_innings","bowling_team_name","innings_type","batsman","bowler","non_striker","super_over",\
"batting_experience","bowler_experience","out_batsman"]].copy()
df4.head()

,venue,current_innings,bowling_team_name,innings_type,batsman,bowler,non_striker,super_over,batting_experience,bowler_experience,out_batsman
0,Karachi,Pakistan,West Indies,score_setter,Fakhar Zaman,S Badree,Babar Azam,No,High,High,None
1,Karachi,Pakistan,West Indies,score_setter,Fakhar Zaman,S Badree,Babar Azam,No,High,High,None
2,Karachi,Pakistan,West Indies,score_setter,Fakhar Zaman,S Badree,Babar Azam,No,High,High,None
3,Karachi,Pakistan,West Indies,score_setter,Fakhar Zaman,S Badree,Babar Azam,No,High,High,None
4,Karachi,Pakistan,West Indies,score_setter,Fakhar Zaman,S Badree,Babar Azam,No,High,High,None


In [5]:
df3 = pd.read_csv("/home/shivargha/cricket_analytics/cricket_score_simulator/Final_T20_trainable.csv")
df3.head(2)

,venue_0,venue_1,venue_2,venue_3,venue_4,venue_5,venue_6,venue_7,current_innings_0,current_innings_1,...,bowler_prop6,bowler_prop7,bowler_prop8,explosivity,running_rating,power_play_rating,end_over_explosivity,wicket_taking_rating,bowling_consistency_rating,outcome
0,0,0,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.169118,0.420956,0.189509,0.195402,0.052496,19.016393,0
1,0,0,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.169118,0.420956,0.189509,0.195402,0.052496,19.016393,0


In [8]:
print(list(df3.columns))

['venue_0', 'venue_1', 'venue_2', 'venue_3', 'venue_4', 'venue_5', 'venue_6', 'venue_7', 'current_innings_0', 'current_innings_1', 'current_innings_2', 'current_innings_3', 'current_innings_4', 'current_innings_5', 'current_innings_6', 'bowling_team_name_0', 'bowling_team_name_1', 'bowling_team_name_2', 'bowling_team_name_3', 'bowling_team_name_4', 'bowling_team_name_5', 'bowling_team_name_6', 'innings_type_0', 'innings_type_1', 'batsman_0', 'batsman_1', 'batsman_2', 'batsman_3', 'batsman_4', 'batsman_5', 'batsman_6', 'batsman_7', 'batsman_8', 'batsman_9', 'batsman_10', 'batsman_11', 'bowler_0', 'bowler_1', 'bowler_2', 'bowler_3', 'bowler_4', 'bowler_5', 'bowler_6', 'bowler_7', 'bowler_8', 'bowler_9', 'bowler_10', 'non_striker_0', 'non_striker_1', 'non_striker_2', 'non_striker_3', 'non_striker_4', 'non_striker_5', 'non_striker_6', 'non_striker_7', 'non_striker_8', 'non_striker_9', 'non_striker_10', 'non_striker_11', 'super_over_0', 'super_over_1', 'batting_experience_0', 'batting_exper

In [9]:
concat_df = pd.concat([df4,df3],axis=1)
concat_df.to_csv("/home/shivargha/cricket_analytics/cricket_score_simulator/categories_with_encoding.csv",index=False)

In [22]:
len(concat_df[concat_df["outcome"]==8])

21972

In [12]:
for i in df3.columns:
    #print(i)
    print(i,min(df3[i]),max(df3[i]))

venue_0 0 1
venue_1 0 1
venue_2 0 1
venue_3 0 1
venue_4 0 1
venue_5 0 1
venue_6 0 1
venue_7 0 1
current_innings_0 0 1
current_innings_1 0 1
current_innings_2 0 1
current_innings_3 0 1
current_innings_4 0 1
current_innings_5 0 1
current_innings_6 0 1
bowling_team_name_0 0 1
bowling_team_name_1 0 1
bowling_team_name_2 0 1
bowling_team_name_3 0 1
bowling_team_name_4 0 1
bowling_team_name_5 0 1
bowling_team_name_6 0 1
innings_type_0 0 1
innings_type_1 0 1
batsman_0 0 1
batsman_1 0 1
batsman_2 0 1
batsman_3 0 1
batsman_4 0 1
batsman_5 0 1
batsman_6 0 1
batsman_7 0 1
batsman_8 0 1
batsman_9 0 1
batsman_10 0 1
batsman_11 0 1
bowler_0 0 1
bowler_1 0 1
bowler_2 0 1
bowler_3 0 1
bowler_4 0 1
bowler_5 0 1
bowler_6 0 1
bowler_7 0 1
bowler_8 0 1
bowler_9 0 1
bowler_10 0 1
non_striker_0 0 1
non_striker_1 0 1
non_striker_2 0 1
non_striker_3 0 1
non_striker_4 0 1
non_striker_5 0 1
non_striker_6 0 1
non_striker_7 0 1
non_striker_8 0 1
non_striker_9 0 1
non_striker_10 0 1
non_striker_11 0 1
super_over_0